In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from gymnasium.wrappers import TimeLimit
from env_hiv import HIVPatient
import random
import gymnasium as gym
from tqdm.notebook import tqdm

In [7]:
env = TimeLimit(
    env=HIVPatient(domain_randomization=False), max_episode_steps=200
)  # The time wrapper limits the number of steps in an episode at 200.
# Now is the floor is yours to implement the agent and train it.

In [9]:
class ReplayBuffer:
    def __init__(self, capacity, device):
        self.capacity = capacity # capacity of the buffer
        self.data = []
        self.index = 0 # index of the next cell to be filled
        self.device = device
    def append(self, s, a, r, s_, d):
        if len(self.data) < self.capacity:
            self.data.append(None)
        self.data[self.index] = (s, a, r, s_, d)
        self.index = (self.index + 1) % self.capacity
    def sample(self, batch_size):
        batch = random.sample(self.data, batch_size)
        return list(map(lambda x:torch.Tensor(np.array(x)).to(self.device), list(zip(*batch))))
    def __len__(self):
        return len(self.data)
    
def greedy_action(network, state):
    device = "cuda" if next(network.parameters()).is_cuda else "cpu"
    with torch.no_grad():
        Q = network(torch.Tensor(state).unsqueeze(0).to(device))
        return torch.argmax(Q).item()

In [39]:
# You have to implement your own agent.
# Don't modify the methods names and signatures, but you can add methods.
# ENJOY!
class ProjectAgent:

    def __init__(self, config, model, device):

        self.gamma = config["gamma"]
        self.batch_size = config["batch_size"]
        self.nb_actions = config["nb_actions"]
        self.memory = ReplayBuffer(config["buffer_size"], device)
        self.epsilon_max = config["epsilon_max"]
        self.epsilon_min = config["epsilon_min"]
        self.epsilon_stop = config["epsilon_decay_period"]
        self.epsilon_delay = config["epsilon_delay_decay"]
        self.epsilon_step = (self.epsilon_max - self.epsilon_min) / self.epsilon_stop
        self.model = model
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=config["learning_rate"]
        )

        pass

    def act(self, observation, use_random=False):

        action = greedy_action(self.model, observation)

        return 0

    def save(self, path):
        pass

    def load(self):
        pass

    def gradient_step(self):
        if len(self.memory) > self.batch_size:
            X, A, R, Y, D = self.memory.sample(self.batch_size)
            QYmax = self.model(Y).max(1)[0].detach()
            # update = torch.addcmul(R, self.gamma, 1-D, QYmax)
            update = torch.addcmul(R, 1 - D, QYmax, value=self.gamma)
            QXA = self.model(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss = self.criterion(QXA, update.unsqueeze(1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def train(self, env, max_episode):
        episode_return = []
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0

        with tqdm(total=100) as pbar:
            while episode < max_episode:
                pbar.update(10)
                # update epsilon
                if step > self.epsilon_delay:
                    epsilon = max(self.epsilon_min, epsilon - self.epsilon_step)

                # select epsilon-greedy action
                if np.random.rand() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = greedy_action(self.model, state)

                # step
                next_state, reward, done, trunc, _ = env.step(action)
                self.memory.append(state, action, reward, next_state, done)
                episode_cum_reward += reward

                # train
                self.gradient_step()

                # next transition
                step += 1
                if done or trunc:
                    episode += 1
                    print(
                        "Episode ",
                        "{:3d}".format(episode),
                        ", epsilon ",
                        "{:6.2f}".format(epsilon),
                        ", batch size ",
                        "{:5d}".format(len(self.memory)),
                        ", episode return ",
                        "{:4.1f}".format(episode_cum_reward),
                        sep="",
                    )
                    state, _ = env.reset()
                    episode_return.append(episode_cum_reward)
                    episode_cum_reward = 0
                else:
                    state = next_state

        return episode_return

In [40]:
# Declare network
state_dim = env.observation_space.shape[0]
n_action = env.action_space.n
nb_neurons = 24
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DQN config
config = {
    "nb_actions": n_action,
    "learning_rate": 1e-4,
    "gamma": 0.95,
    "buffer_size": 1000000,
    "epsilon_min": 0.01,
    "epsilon_max": 1.0,
    "epsilon_decay_period": 1000,
    "epsilon_delay_decay": 20,
    "batch_size": 20,
}

DQN = torch.nn.Sequential(
    nn.Linear(state_dim, nb_neurons),
    nn.ReLU(),
    nn.Linear(nb_neurons, nb_neurons),
    nn.ReLU(),
    nn.Linear(nb_neurons, n_action),
).to(device)


In [41]:
# Train agent
agent = ProjectAgent(config, DQN, device=device)
env.reset()
scores = agent.train(env, max_episode=200)
plt.plot(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

Episode   1, epsilon   0.82, batch size   200, episode return 11292609.8
Episode   2, epsilon   0.62, batch size   400, episode return 10298372.7
Episode   3, epsilon   0.43, batch size   600, episode return 9000151.1
Episode   4, epsilon   0.23, batch size   800, episode return 6375402.7
Episode   5, epsilon   0.03, batch size  1000, episode return 4140289.0
Episode   6, epsilon   0.01, batch size  1200, episode return 3696070.8
Episode   7, epsilon   0.01, batch size  1400, episode return 3671146.9
Episode   8, epsilon   0.01, batch size  1600, episode return 3652153.3
Episode   9, epsilon   0.01, batch size  1800, episode return 3672928.9


KeyboardInterrupt: 